In [1]:
from scipy import sparse
import numpy as np
import literature

R = sparse.load_npz("data/thrm_vertex_matrix.npz")
mats = np.array(open("data/thrm_mats.txt", "r").read().splitlines())
props = ["thermoelectric"]

In [2]:
yrs = np.loadtxt('data/thrm_years.txt')
# We'll consider papers that are published in range [1996, 2000]
R = R[(yrs>=1996)*(yrs<=2000),:]

In [3]:
h = literature.hypergraph(R, mats, props)

In [4]:
length = 20                 # length of the walk
size = 1                    # number of the walk
prop_ind = R.shape[1]-1     # column index of the property as the starting node 

In [5]:
h.random_walk(length, size, start_inds=prop_ind, rand_seed=0)    # uniform sampling

# resulting in the following output: 
# (the first array is the sequence of selected nodes; the second array is the selected papers along the walk):
# ---------------------
# (['thermoelectric a_1244326 a_1084770 a_1085357 CoCrFeMnNi a_281555 a_1076970 CSi a_10764 Al2O3
# K2O a_1672448 CaF2 a_460834 BaF2 a_638548 a_1287239 a_955446 a_955445 a_955447'],
#  ['962469 1191497 746280 1191497 1421491 734403 1115449 132804 46832 1194889 1400463 1400463 23
# 2314 232314 894012 1035899 1035899 615755 1075096'])

(['thermoelectric CeGeNi a_886058 a_885358 Al10Ce10NiPd9 a_886058 a_885610 thermoelectric a_425231 a_524526 thermoelectric a_1710042 thermoelectric a_815535 KO5PTi K2O Na2O GeO2 a_99281 GeO2'],
 ['50739 50739 50737 50729 50729 52319 50739 9151 9151 9151 83553 83553 18658 18633 49949 19121 36848 646 646'])

In [6]:
with open("rw_seqs.txt", "w") as file:
    for i in range(100):
        rw_seqs = h.random_walk(length, size, start_inds=prop_ind, alpha=1, rand_seed=i)[0][0]    # non-uniform sampling (alpha=1)
        file.write(rw_seqs+'\n')

In [7]:
import utils 
seqs = open("rw_seqs.txt").read().splitlines()                              # reading the sequences
seqs_noauthors = utils.remove_authors_from_RW(seqs)                         # removing the author nodes
open("rw_seqs_noauthors.txt", "w").write("\n".join(seqs_noauthors)+"\n")    # saving the pruned sequences

5587

In [8]:
seqs_noauthor_path = "rw_seqs_noauthors.txt"

import embedding
embed = embedding.dww2v(seqs_noauthor_path, workers=20)     # initiating deepwalk model with a different value for parameter workers
embed.build_model()
embed.train()

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject